In [1]:
import cobra
import pandas as pd
import modelseedpy
%run analysis.py

modelseedpy 0.4.0


In [2]:
model_rhoda = cobra.io.load_json_model('./data/model_rhoda_gf2.json')
model_acido = cobra.io.load_json_model('./data/model_acido_gf2.json')

In [3]:
model_acido.reactions.rxn10126_c0.upper_bound = 0
model_rhoda.reactions.rxn10126_c0.upper_bound = 0

In [4]:
model_rhoda.reactions.rxn10577_c0.upper_bound = 0
model_rhoda.reactions.rxn11937_c0.lower_bound = 0

In [5]:
model_rhoda.metabolites.cpd00528_e0

Metabolite identifier,cpd00528_e0
Name,N2 [e0]
Memory address,0x118e98f70
Formula,N2
Compartment,e0
In 4 reaction(s),"rxn11937_c0, EX_cpd00528_e0, dnr00004_c0, rxn10577_c0"


In [6]:
%run ms_ext_symcom.py
cf = CommFactory().with_model(model_acido, 0.7, 'A').with_model(model_rhoda, 0.3, 'R')
model_comm = _build(cf)
if 'bio1' not in model_comm.reactions:
    r_bio_sum = Reaction("bio1", "bio_com", "", 0, 1000)
    r_bio_sum.add_metabolites(
        {
            model_comm.metabolites.cpd11416_cA: -0.4,
            model_comm.metabolites.cpd11416_cR: -0.6,
        }
    )
    model_comm.add_reactions([r_bio_sum])
model_comm.objective = 'bio1'
model_comm

Name,model
Memory address,11a701210
Number of metabolites,2401
Number of reactions,2596
Number of genes,0
Number of groups,0
Objective expression,1.0*bio1 - 1.0*bio1_reverse_b18f7
Compartments,"cA, e0, cR"


In [7]:
GSP_MEDIUM = {
    'EX_cpd00001_e0': 1000.0,
    'EX_cpd00013_e0': 1000.0,

    'EX_cpd00209_e0': 12.0,
    'EX_cpd00029_e0': 20.0, # acetate
 
    'EX_cpd00218_e0': 100.0,
    'EX_cpd00220_e0': 100.0,
    'EX_cpd00644_e0': 0.0002281,
    'EX_cpd00305_e0': 100.0,
    'EX_cpd00393_e0': 100.0,
    'EX_cpd03424_e0': 100.0,
    'EX_cpd00443_e0': 100.0,
    'EX_cpd00263_e0': 100.0,
    'EX_cpd00048_e0': 100.0,
    'EX_cpd00009_e0': 100.0,
    'EX_cpd00242_e0': 29.759425,
    'EX_cpd00205_e0': 1.3415688,
    'EX_cpd00063_e0': 100.0,
    'EX_cpd00971_e0': 34.9324073,
    'EX_cpd00099_e0': 100.0,
    'EX_cpd00254_e0': 100.0,
    'EX_cpd00030_e0': 100.0,
    'EX_cpd00058_e0': 100.0,
    'EX_cpd00034_e0': 100.0,
    'EX_cpd10515_e0': 100.0,
    'EX_cpd00149_e0': 100.0,
    'EX_cpd00244_e0': 100.0,
    'EX_cpd11574_e0': 100.0,
    'EX_cpd15574_e0': 100.0,
    'EX_cpd00067_e0': 100.0,
 }
model_comm.medium = {rxn_id: v for rxn_id, v in GSP_MEDIUM.items() if rxn_id in model_comm.reactions}

In [8]:
#model_comm.reactions.LeuE_cA.upper_bound = -10
solution_comm_wt = cobra.flux_analysis.pfba(model_comm)
model_comm.summary(solution_comm_wt)

Metabolite,Reaction,Flux,C-Number,C-Flux
cpd00009_e0,EX_cpd00009_e0,0.1719,0,0.00%
cpd00013_e0,EX_cpd00013_e0,2.081,0,0.00%
cpd00029_e0,EX_cpd00029_e0,12.57,2,99.98%
cpd00030_e0,EX_cpd00030_e0,0.0004162,0,0.00%
cpd00034_e0,EX_cpd00034_e0,0.0004162,0,0.00%
cpd00048_e0,EX_cpd00048_e0,0.05345,0,0.00%
cpd00058_e0,EX_cpd00058_e0,0.0004162,0,0.00%
cpd00063_e0,EX_cpd00063_e0,0.0004162,0,0.00%
cpd00067_e0,EX_cpd00067_e0,22.86,0,0.00%
cpd00099_e0,EX_cpd00099_e0,0.0004162,0,0.00%


In [9]:
ignore = {
    'EX_cpd00067_e0', # H+ [e0]
    'EX_cpd00058_e0', # Cu2+ [e0]
    'EX_cpd00205_e0', # K+ [e0]
    'EX_cpd00254_e0', # Mg [e0]
    'EX_cpd00034_e0', # Zn2+ [e0]
    'EX_cpd00030_e0', # Mn2+ [e0]
    'EX_cpd00063_e0', # Ca2+ [e0]
    'EX_cpd00099_e0', # Cl- [e0]
    'EX_cpd10515_e0', # Fe2+ [e0]
    'EX_cpd00001_e0', # H2O [e0]
    'EX_cpd00009_e0', # Phosphate [e0]
    'EX_cpd00305_e0', # Thiamin [e0]
    'EX_cpd00048_e0', # Sulfate [e0]
}

In [10]:
for ex in model_comm.exchanges:
    #if ex.id not in ignore:
    m = list(ex.metabolites)[0]
    print(m.id, m.name)
    for r in m.reactions:
        if r != ex:
            print('\t', r.id, r.build_reaction_string(True))

cpd00067_e0 H+ [e0]
	 rxn05317_cR H+ [e0] + Deoxyadenosine [e0] <=> H+ [R] + Deoxyadenosine [R]
	 rxn05659_cA H+ [e0] + Thymine [e0] <=> H+ [A] + Thymine [A]
	 dnr00004_cA 2 H+ [e0] + Nitrous oxide [e0] + 2 Reduced azurin [A] --> H2O [e0] + N2 [e0] + 2 Oxidized azurin [A]
	 rxn05312_cA Phosphate [e0] + H+ [e0] <-- Phosphate [A] + H+ [A]
	 rxn08323_cR H+ [e0] + Deoxyguanosine [e0] <=> H+ [R] + Deoxyguanosine [R]
	 rxn05514_cA Ca2+ [A] + H+ [e0] <-- Ca2+ [e0] + H+ [A]
	 rxn05500_cR H+ [e0] + L-Arabinose [e0] <=> H+ [R] + L-Arabinose [R]
	 rxn14415_cA 0.5 O2 [A] + 4 H+ [A] + 3 Cytochrome c2+ [A] --> H2O [A] + 2 H+ [e0] + 3 Cytochrome c3+ [A]
	 rxn05561_cA H+ [e0] + Fumarate [e0] <=> H+ [A] + Fumarate [A]
	 rxn05316_cR H+ [e0] + Inosine [e0] <=> H+ [R] + Inosine [R]
	 rxn10161_cR H+ [e0] + ocdca [e0] <=> H+ [R] + ocdca [R]
	 rxn08722_cR H+ [e0] + L-Homoserine [R] --> H+ [R] + L-Homoserine [e0]
	 rxn14427_cR 3 H+ [e0] + Cytochrome c2+ [R] + Nitrate [R] --> H2O [R] + Nitrite [R] + Cytochrome

In [11]:
model_comm

Name,model
Memory address,11a701210
Number of metabolites,2401
Number of reactions,2596
Number of genes,0
Number of groups,0
Objective expression,1.0*bio1 - 1.0*bio1_reverse_b18f7
Compartments,"cA, e0, cR"


In [12]:
test_reactions = []
for m in model_comm.metabolites:
    if m.id.endswith('cA'):
        other = m.id[:-1] + 'R'
        if other in model_comm.metabolites:
            m_other = model_comm.metabolites.get_by_id(other)
            import cobra.core.reaction
            rxn_test = Reaction(f'bridge_{m.id}', f'bridge {m.name}', 'test', 0, 0)
            rxn_test.add_metabolites({
                m: -1,
                m_other: 1
            })
            test_reactions.append(rxn_test)
            print(m.id, m_other.id)
model_comm.add_reactions(test_reactions)

cpd00443_cA cpd00443_cR
cpd02920_cA cpd02920_cR
cpd00012_cA cpd00012_cR
cpd00067_cA cpd00067_cR
cpd00683_cA cpd00683_cR
cpd00002_cA cpd00002_cR
cpd00033_cA cpd00033_cR
cpd00506_cA cpd00506_cR
cpd00008_cA cpd00008_cR
cpd00009_cA cpd00009_cR
cpd00042_cA cpd00042_cR
cpd00022_cA cpd00022_cR
cpd00054_cA cpd00054_cR
cpd00010_cA cpd00010_cR
cpd00722_cA cpd00722_cR
cpd00046_cA cpd00046_cR
cpd00096_cA cpd00096_cR
cpd00106_cA cpd00106_cR
cpd00037_cA cpd00037_cR
cpd03494_cA cpd03494_cR
cpd00014_cA cpd00014_cR
cpd03495_cA cpd03495_cR
cpd00001_cA cpd00001_cR
cpd00658_cA cpd00658_cR
cpd00020_cA cpd00020_cR
cpd02566_cA cpd02566_cR
cpd11440_cA cpd11440_cR
cpd00018_cA cpd00018_cR
cpd11441_cA cpd11441_cR
cpd00023_cA cpd00023_cR
cpd00084_cA cpd00084_cR
cpd00004_cA cpd00004_cR
cpd11527_cA cpd11527_cR
cpd00003_cA cpd00003_cR
cpd00533_cA cpd00533_cR
cpd00356_cA cpd00356_cR
cpd00075_cA cpd00075_cR
cpd00851_cA cpd00851_cR
cpd00213_cA cpd00213_cR
cpd03049_cA cpd03049_cR
cpd00056_cA cpd00056_cR
cpd00836_cA cpd0

In [13]:
flux_bio1_wt = solution_comm_wt.fluxes['bio1']
flux_bio1_wt

np.float64(0.2601030816317411)

In [24]:
rxn_test

Reaction identifier,bridge_cpd01772_cA
Name,bridge Succinylbenzoate [A]
Memory address,0x11bec1b10
Stoichiometry,cpd01772_cA --> cpd01772_cR Succinylbenzoate [A] --> Succinylbenzoate [R]
GPR,
Lower bound,0
Upper bound,0


In [23]:
from tqdm import tqdm
with open('ex_test.tsv', 'w') as fh:
    for t in tqdm(test_reactions):
        rxn_test = model_comm.reactions.get_by_id(t.id)
        rxn_test.lower_bound = -1000
        rxn_test.upper_bound = 1000
        m = list(rxn_test.metabolites)[0]
        solution_comm_test = cobra.flux_analysis.pfba(model_comm)
        flux_bio1 = solution_comm_test.fluxes['bio1']
        row = [rxn_test.id, rxn_test.name, m.name, m.id, m.formula, m.charge, m.elements.get('C', 0), 
               solution_comm_test[rxn_test.id], flux_bio1, flux_bio1_wt - flux_bio1]
        #print(rxn_test.name, solution_comm_test[rxn_test.id], flux_bio1, flux_bio1_wt - flux_bio1)
        fh.write('\t'.join([str(x) for x in row]) + '\n')
        rxn_test.lower_bound = 0
        rxn_test.upper_bound = 0

100%|████████████████████████████████████████████████████████████████████████████████████████| 1013/1013 [52:09<00:00,  3.09s/it]


In [65]:
import cobra.core.reaction
rxn = Reaction('bridge_pyr', 'bridge pyr', 'test', -1000, 1000)
rxn.add_metabolites({
    model_comm.metabolites.cpd00020_cA: -1,
    model_comm.metabolites.cpd00020_cR: 1
})
model_comm.add_reactions([rxn])

In [60]:
model_comm.reactions.rxn05559_cR.lower_bound = -1000
model_comm.reactions.rxn05559_cR.upper_bound = 1000
model_comm.reactions.rxn05559_cR

Reaction identifier,rxn05559_cR
Name,formate transport in via proton symport [R]
Memory address,0x126e2bb80
Stoichiometry,cpd00047_e0 + cpd00067_e0 <=> cpd00047_cR + cpd00067_cR Formate [e0] + H+ [e0] <=> Formate [R] + H+ [R]
GPR,
Lower bound,-1000
Upper bound,1000


In [40]:
for ex in model_comm.exchanges:
    if ex.id not in ignore:
        v = solution_comm_wt.fluxes[ex.id]
        if v != 0:
            m = list(ex.metabolites)[0]
            print(v, ex.id, m.name)
            for rxn in m.reactions:
                rxn_v = solution_comm_wt.fluxes[rxn.id]
                if rxn_v != 0 and rxn != ex:
                    print('\t', rxn_v, rxn.id)

-1.0125233984581428e-13 EX_cpd00075_e0 Nitrite [e0]
	 12.0 rxn09004_cA
	 12.000000000000101 dnr00002_cR
-2.081427266265087 EX_cpd00013_e0 NH3 [e0]
	 0.8325709065060323 rxn05466_cA
	 1.248856359759055 rxn05466_cR
-12.0 EX_cpd00209_e0 Nitrate [e0]
	 12.0 rxn09004_cA
6.000000000000051 EX_cpd00528_e0 N2 [e0]
	 6.000000000000051 dnr00004_cR
-0.0004161755070368777 EX_cpd00149_e0 Co2+ [e0]
	 -0.0002497053042221266 rxn08241_cR
	 -0.0001664702028147511 rxn08241_cA
-12.574919157734755 EX_cpd00029_e0 Acetate [e0]
	 5.525709117571984 rxn05488_cR
	 7.0492100401627695 rxn05488_cA
15.90602401068816 EX_cpd00011_e0 CO2 [e0]
	 -5.01761307522574 rxn05467_cA
	 -10.88841093546242 rxn05467_cR
0.0001664702028147511 EX_cpd01981_e0 5-Methylthio-D-ribose [e0]
	 -0.0001664702028147511 rxn05481_cA


Reaction identifier,rxn05559_cR
Name,formate transport in via proton symport [R]
Memory address,0x126e2bb80
Stoichiometry,cpd00047_e0 + cpd00067_e0 <=> cpd00047_cR + cpd00067_cR Formate [e0] + H+ [e0] <=> Formate [R] + H+ [R]
GPR,
Lower bound,-1000.0
Upper bound,1000.0


Metabolite,Reaction,Flux,C-Number,C-Flux
cpd00009_e0,EX_cpd00009_e0,0.1719,0,0.00%
cpd00013_e0,EX_cpd00013_e0,2.081,0,0.00%
cpd00029_e0,EX_cpd00029_e0,12.57,2,99.98%
cpd00030_e0,EX_cpd00030_e0,0.0004162,0,0.00%
cpd00034_e0,EX_cpd00034_e0,0.0004162,0,0.00%
cpd00048_e0,EX_cpd00048_e0,0.05345,0,0.00%
cpd00058_e0,EX_cpd00058_e0,0.0004162,0,0.00%
cpd00063_e0,EX_cpd00063_e0,0.0004162,0,0.00%
cpd00067_e0,EX_cpd00067_e0,22.86,0,0.00%
cpd00099_e0,EX_cpd00099_e0,0.0004162,0,0.00%
